# Optimization of Training Selections Using PuLP

<i>What you can learn from this post:
- How to formulate a simple optimization into mathematical formulas
- Using the Python PuLP package to solve a simple optimization
- An analytical approach we use to organize an internal event
</i>


Target audience: technical, but learning (like us!)


## Problem Introduction

Our Analytics department places high value on continuous learning and development. That's why we decided that our 2015 department summit would include break-out training sessions as a part of the fun. But how would we best decide which fun and exciting things to learn more about, and who to assign to each training? Well, this seemed like a fun sample problem for learning about optimization!

## Gathering Data

The first step was to design a survey. The planning committee identified six potential training topics that could be prepared by people in our department, of which we would choose three based on the survey results:

* Model Statistics with R (R Stats)
* Introduction to Simulation (Sim)
* Using Public APIs (APIs)
* Introduction to Python (Python)
* Data Visualization (Viz)
* Introduction to NLP (NLP)

We used Survey Monkey to create a simple survey asking everyone in the department to rank their preferences (or check "Not Interested"). 

![survey image - link not working](survey_choices_image.png)

We manually extracted the survey results to create a csv file with one row per person (30 total rows) with the rank for each course (or NI for "Not Interested"):

ID |Name  | R Stats | Sim | APIs | Python | Viz | NLP
---|----- | ------- | --- | ---- | ------ | --- | 
01 |Alice | 1       | 4   | NI   | 2      | NI  | 3
02 |Bob   | 4       | 6   | 3    | 2      | 1   | 5
...|...   | ...       | ...   | ...    | ...      | ...   | ...


## Determining Our Model

In the ideal world, everyone would be assigned to their first choice course, but with the variety of interests across our group, this was not be possible. In the absense of giving everyone their first choice, we would instead attempt to assign everyone to their most prefered course given a set of contraints. This gives us the objective of the optimization, which can be written in mathy form as:

$$ 
    \\
    \underset{x,y}{\text{minimize}} 
    \sum\limits_{workshops} \sum\limits_{students} preference \centerdot x_{student,workshop} \\
$$
where <i>x<sub>s,w</sub></i> is a binary (0 or 1) decision variable to decide whether student <i>s</i> is assigned to workshop <i>w</i>. Note this is a minimization because lower rank indicates more preference (i.e. rank=1 indicates the student's top choice). 

The first substantive constraint we want to add is to limit the number of open workshops to three. To do this, we will add another binary variable <i>y<sub>w</sub></i> to indicate whether workshop w is open (1) or closed (0). Our constraints to ensure we have only three workshops are:

$$
    \ Total \ workshops: \ \ \ \sum\limits_{workshops} y_{workshop} = 3 \\
    \ Only \ valid \ workshops: \ \ \ x_{student,workshop} \leq y_{workshop} \ \ \forall \ \ students, workshops \\
$$

The next constraint we want to address is the number of students per workshop. We don't want to have one of the workshop leaders preparing a course for just one or two students. To ensure things are even, we set a lower bound on the number of students per open workshop to seven, as follows:

$$
    \ Min \ students \ per \ workshop: \ \ \ \sum\limits_{students} x_{student,workshop} \geq 7 y_{workshop} \ \ \forall \ \ workshops \\
$$

The remaining constraints are things that are common sense to us, but might not be common sense to the computer. First, we need to tell the computer that <i>x</i> and <i>u</i> are binary variables, as mentioned earlier:

$$
    \ x_{student,workshop} \ , \ y_{workshop} \ \ \in \ \ \{0,1\} \ \ \forall \ \ students, workshops \\ 
$$

And lastly, we want to make sure that each student is attending exactly one workshop:

$$
    \sum\limits_{workshops} x_{student,workshop} = 1 \ \ \forall \ \ students \\
$$

The full model is summarized below.

### Final Model

Objective:

$$ 
    \\
    \underset{x,y}{\text{minimize}} 
    \sum\limits_{workshops} \ \sum\limits_{students} preference \centerdot x_{student,workshop} \\
$$

Subject to:
$$
    (1) \ Binary \ variables: \ \ \ x_{student,workshop} \ , \ y_{workshop} \ \ \in \ \ \{0,1\} \ \ \forall \ \ students, workshops \\
    (2) \ Total \ workshops: \ \ \ \sum\limits_{workshops} y_{workshop} = 3 \\
    (3) \ Workshops \ per \ student: \ \ \ \sum\limits_{workshops} x_{student,workshop} = 1 \ \ \forall \ \ students \\
    \\
    (4) \ Min \ students \ per \ workshop: \ \ \ \sum\limits_{students} x_{student,workshop} \geq 7 y_{workshop} \ \ \forall \ \ workshops \\
    \\
    (5) \ Only \ valid \ workshops: \ \ \ x_{student,workshop} \leq y_{workshop} \ \ \forall \ \ students, workshops \\
$$

## Data Cleaning

We have a model, we have data - are we ready to start coding? Not quite. Before we can start coding, we have a little data cleaning to do.

Remember those pesky NI's in our dataset? Our code will not know how to handle those in numeric fields. Since our inclination is to not put anyone into a workshop that they have no interest in, we can replace these NI entries with some big number. We chose 100. For our previous sample data, that update resulted in:


ID |Name  | R Stats | Sim | APIs | Python | Viz | NLP
---|----- | ------- | --- | ---- | ------ | --- | 
01 |Alice | 1       | 4   | 100  | 2      | 100 | 3
02 |Bob   | 4       | 6   | 3    | 2      | 1   | 5
...|...   | ...       | ...   | ...    | ...      | ...   | ...

Another complication is that these workshops would be prepared by people in our department - meaning, that there are six workshop-person combinations that are actually that person ranking their interest in taking their own workshop. To simplify, and ensure that all workshop leaders are assigned to their own workshop (if it is open), we gave a rank of 0 to any workshop that a person was the leader of. For example, if Alice was the potential instructor for the Viz workshop, that update would result in:

ID |Name  | R Stats | Sim | APIs | Python | Viz | NLP
---|----- | ------- | --- | ---- | ------ | --- | 
01 |Alice | 1       | 4   | 0    | 2      | 100 | 3
02 |Bob   | 4       | 6   | 3    | 2      | 1   | 5
...|...   | ...       | ...   | ...    | ...      | ...   | ...

## Optimization Using Python and PuLP

The following subsections walk you through how we used Python to solve the optimization. You can skip this if code isn't your thing :)

Note: This was probably the first code I wrote in Python, so if you are a real user of Python, you will probably notice that this code is not very efficient. Please excuse my lack of expertise in this area! 

### Read Input Data

The following Python code reads the csv and prepares the data in the format we will need to use PuLP. The print statements at the end of this section show samples of these inputs. 

In [1]:
#reading files

# raw file
source_csv = 'workshop_survey_clean.csv'
min_class_size = 7

import numpy as np
csv = np.genfromtxt(source_csv, delimiter=",")
csv_rows = len(csv)

preferences = np.zeros((len(csv)-1,6))

# gets preference array
for x in range(1,len(csv)):
    preferences[x-1] =csv[x][2:8]

nstudents=preferences.shape[0]
nworkshops=preferences.shape[1]


rstudents = range(0,nstudents)
rworkshops = range(0,nworkshops)


import csv
with open(source_csv) as csvfile:
    reader = csv.DictReader(csvfile)
    workshop_names = reader.fieldnames[2:8]
    student_names = [row['name'] for row in reader]

print("Number of students: ", nstudents)
print()
print("Number of workshops: ", nworkshops)
print()
print("Minimum workshop size: ", min_class_size)
print()
print("Workshop names: ", workshop_names)
print()
print("Student names (sample): ", student_names[1:5])
print()
print("Preferences matrix (sample): ")
print(preferences[0:3,:])

Number of students:  30

Number of workshops:  6

Minimum workshop size:  7

Workshop names:  ['R Stats', 'Sim', 'APIs', 'Python', 'Viz', 'NLP']

Student names (sample):  ['Outis', 'Odysseus', 'Tiresias', 'Polyphemus']

Preferences matrix (sample): 
[[ 5.  1.  6.  4.  3.  2.]
 [ 4.  5.  2.  3.  6.  1.]
 [ 6.  5.  4.  1.  2.  3.]]


### Prepare the PuLP Model

Next, we want to prepare the PuLP model. The following code imports the PuLP package and prepares the objective function and constraints. One thing to note about adding constraints to PuLP is that you need to form the constraints so that all variables are on the left side of the inequality. 

In [2]:
##############################
# prep opt model

import pulp

# Create Problem Instance for Minimization
workshop_model = pulp.LpProblem("Workshop Preferences",pulp.LpMinimize)

# Decision Variable For Workshop Choice By Student
placement = pulp.LpVariable.dicts("Workshop Placement",(rstudents,rworkshops),0,1,pulp.LpInteger)

# Decision Variable For Whether Workshop Offered
offered = pulp.LpVariable.dicts("Workshop Offered",(rworkshops),0,1,pulp.LpInteger)

# Objective = sum(preference*placement)
workshop_model += pulp.lpSum( [[(preferences[student][workshop]) * placement[student][workshop]]
                                     for workshop in rworkshops]
                                     for student in rstudents), "Placement Value"

# A constraint ensuring that only one value can be in each square is created
workshop_model += pulp.lpSum(offered[workshop] for workshop in rworkshops) == 3, "Workshop Sum"

# A constraint ensuring one workshop per student
for student in (rstudents):
    workshop_model += pulp.lpSum(placement[student][workshop] for workshop in rworkshops) == 1, ""

# A constraint ensuring only place in valid workshops
# Note: reformatted to keep variables on left side of inequality
for student in rstudents:
    for workshop in rworkshops:
        workshop_model += placement[student][workshop] - offered[workshop] <= 0,""
        

# A constraint on the minimum number of people per workshop
for workshop in rworkshops:
    workshop_model += pulp.lpSum(placement[student][workshop] for student in rstudents) - min_class_size*offered[workshop] >= 0,""

print(workshop_model)

Workshop Preferences:
MINIMIZE
5.0*Workshop_Placement_0_0 + 1.0*Workshop_Placement_0_1 + 6.0*Workshop_Placement_0_2 + 4.0*Workshop_Placement_0_3 + 3.0*Workshop_Placement_0_4 + 2.0*Workshop_Placement_0_5 + 0.5*Workshop_Placement_10_0 + 4.0*Workshop_Placement_10_1 + 100.0*Workshop_Placement_10_2 + 2.0*Workshop_Placement_10_3 + 100.0*Workshop_Placement_10_4 + 1.0*Workshop_Placement_10_5 + 2.0*Workshop_Placement_11_0 + 4.0*Workshop_Placement_11_1 + 5.0*Workshop_Placement_11_2 + 3.0*Workshop_Placement_11_3 + 1.0*Workshop_Placement_11_4 + 6.0*Workshop_Placement_11_5 + 4.0*Workshop_Placement_12_0 + 3.0*Workshop_Placement_12_1 + 5.0*Workshop_Placement_12_2 + 2.0*Workshop_Placement_12_3 + 1.0*Workshop_Placement_12_4 + 6.0*Workshop_Placement_12_5 + 5.0*Workshop_Placement_13_0 + 1.0*Workshop_Placement_13_1 + 2.0*Workshop_Placement_13_2 + 100.0*Workshop_Placement_13_3 + 4.0*Workshop_Placement_13_4 + 3.0*Workshop_Placement_13_5 + 4.0*Workshop_Placement_14_0 + 5.0*Workshop_Placement_14_1 + 1.0*Works

But wait! What is this print out mess we have? At the end of the model creation, I asked Python to print the entire model we had constructed. This means it has printed out the objective ("MINIMIZE ... " up to "SUBJECT TO") and the constraints (after "SUBJECT TO"). The reason there are so many more things printed here than when we formulated the model is that PuLP has expanded the simple formulation we gave it to the detail level. For example, our formulation of the one-workshop-per-student constraint:

$$
    \sum\limits_{workshops} x_{student,workshop} = 1 \ \ \forall \ \ students \\
$$

turns into one constraint per student in the PuLP formulation, and the sum is expanded to explicitly state each workshop for the given student, as follows:
```
_C1: Workshop_Placement_0_0 + Workshop_Placement_0_1 + Workshop_Placement_0_2
 + Workshop_Placement_0_3 + Workshop_Placement_0_4 + Workshop_Placement_0_5
 = 1
 
_C2: Workshop_Placement_1_0 + Workshop_Placement_1_1 + Workshop_Placement_1_2
 + Workshop_Placement_1_3 + Workshop_Placement_1_4 + Workshop_Placement_1_5
 = 1
 
 ...
 ```

### Solving the PuLP Model

Now that the model is formulated, we simple submit a request to solve it. The status shows that the PuLP solver was able to find an optimal solution for this model. 

In [3]:
# The problem is solved using PuLP's choice of Solver
workshop_model.solve()

# The status of the solution is printed to the screen
print( "Status:", pulp.LpStatus[workshop_model.status]     )   
print()

Status: Optimal



Lastly, we can print the resulting workshop-student assignments. 

In [4]:
#############################################
# show solution

for workshop in rworkshops:
    if pulp.value(offered[workshop])==1:
        sum=0
        for student in rstudents:
            sum+=pulp.value(placement[student][workshop])
        print( "Workshop ", workshop, workshop_names[workshop],", ", sum, " assigned students:")
        for student in rstudents:
            if pulp.value(placement[student][workshop])==1:
                print( "    ", student_names[student], "(Choice: ", preferences[student][workshop],")")

Workshop  1 Sim ,  11.0  assigned students:
     Eupithes (Choice:  1.0 )
     Calypso (Choice:  2.0 )
     Eumaeus (Choice:  1.0 )
     Melanthius (Choice:  1.0 )
     Laertes (Choice:  1.0 )
     Orestes  (Choice:  1.0 )
     Circe (Choice:  3.0 )
     Penelope (Choice:  2.0 )
     Poseiden (Choice:  0.0 )
     Hermes  (Choice:  1.0 )
     Siren (Choice:  1.0 )
Workshop  3 Python ,  9.0  assigned students:
     Outis (Choice:  3.0 )
     Odysseus (Choice:  1.0 )
     Tiresias (Choice:  2.0 )
     Polyphemus (Choice:  2.0 )
     Athena (Choice:  0.0 )
     Anticleia  (Choice:  2.0 )
     Philoetius (Choice:  2.0 )
     Scylla (Choice:  2.0 )
     Agamemnon (Choice:  1.0 )
Workshop  4 Viz ,  10.0  assigned students:
     Alcinous (Choice:  1.0 )
     Menelaus (Choice:  1.0 )
     Charybdis (Choice:  1.0 )
     Nausicaa (Choice:  1.0 )
     Zeus (Choice:  1.0 )
     Melantho  (Choice:  0.0 )
     Telemarchus (Choice:  1.0 )
     Polyphemus  (Choice:  2.0 )
     Eurycleia (Choice:  2.0 )

## Results

The optimal solution from our model was used to place people to workshops optimizing their preference level. Since there are over seven students per course, we can see that all of the students are assigned to their top choice amoung the selected options. 

We should acknowledge that we have used simpler methods to determine which courses to hold. For example, we could have asked people to vote for their top choice only to determine which three workshops to hold before assigning students. This is equivalent to counting the number of times each workshop was assigned a rank of one. In our data, this would lead to picking Sim and Viz, but then there would be a three-way-tie for the last pick between R Stats, API's, and NLP. Python actually has the least number of people indicating it as their top choice, but several people indicating it as their second choice in the final solution. Using optimization, we can see a solution that meets our needs that might not have been obvious otherwise. Plus, this way gave us a reason to learn more about Python, PuLP, and optimization!